# Homework with McDonald's sentiment data

## Imaginary problem statement

McDonald's receives **thousands of customer comments** on their website per day, and many of them are negative. Their corporate employees don't have time to read every single comment, but they do want to read a subset of comments that they are most interested in. In particular, the media has recently portrayed their employees as being rude, and so they want to review comments about **rude service**.

McDonald's has hired you to develop a system that ranks each comment by the **likelihood that it is referring to rude service**. They will use your system to build a "rudeness dashboard" for their corporate employees, so that employees can spend a few minutes each day examining the **most relevant recent comments**.

## Description of the data

Before hiring you, McDonald's used the [CrowdFlower platform](http://www.crowdflower.com/data-for-everyone) to pay humans to **hand-annotate** about 1500 comments with the **type of complaint**. The complaint types are listed below, with the encoding used in the data listed in parentheses:

- Bad Food (BadFood)
- Bad Neighborhood (ScaryMcDs)
- Cost (Cost)
- Dirty Location (Filthy)
- Missing Item (MissingFood)
- Problem with Order (OrderProblem)
- Rude Service (RudeService)
- Slow Service (SlowService)
- None of the above (na)

## Task 1

Read **`mcdonalds.csv`** into a pandas DataFrame and examine it. (It can be found in the **`data`** directory of the course repository.)

- The **policies_violated** column lists the type of complaint. If there is more than one type, the types are separated by newline characters.
- The **policies_violated:confidence** column lists CrowdFlower's confidence in the judgments of its human annotators for that row (higher is better).
- The **city** column is the McDonald's location.
- The **review** column is the actual text comment.

In [1]:
# imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import numpy as np

In [2]:
vect = CountVectorizer()
nb = MultinomialNB()
lr = LogisticRegression()

In [3]:
path = r'C:\Users\mark\Anaconda3\envs\work\data\mcdonalds.csv'
path1 = r'C:\Users\mark\Anaconda3\envs\Dataschool\work\data\mcdonalds.csv'

In [4]:
mcd = pd.read_csv(path)
mcd.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review,Unnamed: 10
0,679455653,False,finalized,3,2/21/15 0:36,RudeService\r\nOrderProblem\r\nFilthy,1.0\r\n0.6667\r\n0.6667,Atlanta,NaN,"I'm not a huge mcds lover, but I've been to be...",NaN
1,679455654,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,Terrible customer service. ŒæI came in at 9:30...,NaN
2,679455655,False,finalized,3,2/21/15 0:26,SlowService\r\nOrderProblem,1.0\r\n1.0,Atlanta,NaN,"First they ""lost"" my order, actually they gave...",NaN
3,679455656,False,finalized,3,2/21/15 0:27,na,0.6667,Atlanta,NaN,I see I'm not the only one giving 1 star. Only...,NaN
4,679455657,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,"Well, it's McDonald's, so you know what the fo...",NaN


In [5]:
#mcd['rude'] = mcd.policies_violated.str.contains('RudeService')

In [6]:
mcd.shape

(1525, 11)

## Task 2

Remove any rows from the DataFrame in which the **policies_violated** column has a **null value**. Check the shape of the DataFrame before and after to confirm that you only removed about 50 rows.

- **Note:** Null values are also known as "missing values", and are encoded in pandas with the special value "NaN". This is distinct from the "na" encoding used by CrowdFlower to denote "None of the above". Rows that contain "na" should **not** be removed.
- **Hint:** [How do I handle missing values in pandas?](https://www.youtube.com/watch?v=fCMrO_VzeL8&list=PL5-da3qGB5ICCsgW1MxlZ0Hq8LL5U3u9y&index=16) explains how to do this.

In [7]:
mcd_na = pd.DataFrame(mcd.dropna(axis= 0, how='any', subset= {'policies_violated'}))
mcd_na.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review,Unnamed: 10
0,679455653,False,finalized,3,2/21/15 0:36,RudeService\r\nOrderProblem\r\nFilthy,1.0\r\n0.6667\r\n0.6667,Atlanta,NaN,"I'm not a huge mcds lover, but I've been to be...",NaN
1,679455654,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,Terrible customer service. ŒæI came in at 9:30...,NaN
2,679455655,False,finalized,3,2/21/15 0:26,SlowService\r\nOrderProblem,1.0\r\n1.0,Atlanta,NaN,"First they ""lost"" my order, actually they gave...",NaN
3,679455656,False,finalized,3,2/21/15 0:27,na,0.6667,Atlanta,NaN,I see I'm not the only one giving 1 star. Only...,NaN
4,679455657,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,"Well, it's McDonald's, so you know what the fo...",NaN


## Task 3

Add a new column to the DataFrame called **"rude"** that is 1 if the **policies_violated** column contains the text "RudeService", and 0 if the **policies_violated** column does not contain "RudeService". The "rude" column is going to be your response variable, so check how many zeros and ones it contains.

- **Hint:** [How do I use string methods in pandas?](https://www.youtube.com/watch?v=bofaC0IckHo&list=PL5-da3qGB5ICCsgW1MxlZ0Hq8LL5U3u9y&index=12) shows how to search for the presence of a substring, and [How do I change the data type of a pandas Series?](https://www.youtube.com/watch?v=V0AWyzVMf54&list=PL5-da3qGB5ICCsgW1MxlZ0Hq8LL5U3u9y&index=13) shows how to convert the boolean results (True/False) to integers (1/0).

In [8]:
mcd_na['rude'] = mcd_na.policies_violated.str.contains('RudeService')

In [9]:
mcd_na.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review,Unnamed: 10,rude
0,679455653,False,finalized,3,2/21/15 0:36,RudeService\r\nOrderProblem\r\nFilthy,1.0\r\n0.6667\r\n0.6667,Atlanta,NaN,"I'm not a huge mcds lover, but I've been to be...",NaN,True
1,679455654,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,Terrible customer service. ŒæI came in at 9:30...,NaN,True
2,679455655,False,finalized,3,2/21/15 0:26,SlowService\r\nOrderProblem,1.0\r\n1.0,Atlanta,NaN,"First they ""lost"" my order, actually they gave...",NaN,False
3,679455656,False,finalized,3,2/21/15 0:27,na,0.6667,Atlanta,NaN,I see I'm not the only one giving 1 star. Only...,NaN,False
4,679455657,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,"Well, it's McDonald's, so you know what the fo...",NaN,True


## Task 4

1. Define X (the **review** column) and y (the **rude** column).
2. Split X and y into training and testing sets (using the parameter **`random_state=1`**).
3. Use CountVectorizer (with the **default parameters**) to create document-term matrices from X_train and X_test.

In [10]:
X = mcd_na.review
y = mcd_na.rude

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state= 1)

In [12]:
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)
print(len(X_train))

1103


## Task 5

Fit a Multinomial Naive Bayes model to the training set, calculate the **predicted probabilites** (not the class predictions) for the testing set, and then calculate the **AUC**. Repeat this task using a logistic regression model to see which of the two models achieves a better AUC.

- **Note:** Because McDonald's only cares about ranking the comments by the likelihood that they refer to rude service, **classification accuracy** is not the relevant evaluation metric. **Area Under the Curve (AUC)** is a more useful evaluation metric for this scenario, since it measures the ability of the classifier to assign higher predicted probabilities to positive instances than to negative instances.
- **Hint:** [Evaluating a classification model](https://github.com/justmarkham/scikit-learn-videos/blob/master/09_classification_metrics.ipynb) explains how to calculate predicted probabilities and AUC, and my [blog post and video](http://www.dataschool.io/roc-curves-and-auc-explained/) explain AUC in-depth.

In [13]:
nb.fit(X_train_dtm, y_train)
lr.fit(X_train_dtm, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
nb_pred_class = nb.predict(X_test_dtm)
lr_pred_class = lr.predict(X_test_dtm)

In [15]:
nb_pred_prob = nb.predict_proba( X_test_dtm)[:,1]
lr_pred_prob = lr.predict_proba(X_test_dtm)

In [16]:
prob = pd.DataFrame({'X_test': X_test,'rude':X_test.str.contains('RudeService'), 
                     'nb_prob':nb.predict_proba(X_test_dtm)[:,1], 
                     'lr_prob':lr.predict_proba(X_test_dtm)[:,1],
                     'nb_pred':nb_pred_class,
                     'lr_pred':lr_pred_class} )
                    
                   
prob.head(100)

,X_test,lr_pred,lr_prob,nb_pred,nb_prob,rude
501,I'm constantly disappointed in this mcdonalds ...,True,0.625357,True,9.992346e-01,False
1197,One of the worst places to come and grab a bit...,False,0.105759,False,2.592430e-01,False
315,BEWARE!!!!! This Mcdonalds tows vehicles like ...,True,0.790119,True,8.578687e-01,False
436,Typical McDonald's. ŒæI like this location bec...,False,0.037434,False,2.676957e-06,False
1018,Unsanitary place! I walked in because I was fi...,False,0.466568,False,2.685485e-01,False
1511,Worst service ever!!!! There was so much trash...,True,0.891768,True,9.974698e-01,False
305,I have to strike this fast food venue from my ...,False,0.000229,False,7.406998e-04,False
264,"Only ordered 4 items: apple pie, bbq Ranch Bur...",False,0.014879,False,1.114061e-09,False
1103,Where else around USC can you get breakfast at...,False,0.237559,False,1.919146e-01,False
1064,It was raining when I went to this McDonald's ...,False,0.108293,False,4.110116e-01,False


In [17]:
print('nb_auc = ', metrics.roc_auc_score(y_test, nb_pred_class))
print('lr_auc = ', metrics.roc_auc_score(y_test, lr_pred_class))

nb_auc =  0.766428230806
lr_auc =  0.725099348275


## Task 6

Using either Naive Bayes or logistic regression (whichever one had a better AUC in the previous step), try **tuning CountVectorizer** using some of the techniques we learned in class. Check the testing set **AUC** after each change, and find the set of parameters that increases AUC the most.

- **Hint:** It is highly recommended that you adapt the **`tokenize_test()`** function from class for this purpose, since it will allow you to iterate quickly through different sets of parameters.

In [18]:
# define a function that accepts a vectorizer and calculates the accuracy
def tokenize_test(vect):
    
    # create document-term matrices using the vectorizer
    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)
    
    # print the number of features that were generated
    print('Features: ', X_train_dtm.shape[1])
    
    # use Multinomial Naive Bayes to predict the star rating
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)
    y_pred_class = nb.predict(X_test_dtm)
    
    # print the accuracy of its predictions
    print('Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))

In [19]:
# test accuracy of nb using default vectorizer
vect = CountVectorizer()
tokenize_test(vect)

Features:  7300
Accuracy:  0.798913043478


In [20]:
vect = CountVectorizer(min_df= 3)
tokenize_test(vect)

Features:  2449
Accuracy:  0.801630434783


## Task 7 (Challenge)

The **city** column might be predictive of the response, but we are not currently using it as a feature. Let's see whether we can increase the AUC by adding it to the model:

1. Create a new DataFrame column, **review_city**, that concatenates the **review** text with the **city** text. One easy way to combine string columns in pandas is by using the [`Series.str.cat()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.cat.html) method. Make sure to use the **space character** as a separator, as well as replacing **null city values** with a reasonable string value (such as 'na').
2. Redefine X as the **review_city** column, and re-split X and y into training and testing sets.
3. When you run **`tokenize_test()`**, CountVectorizer will simply treat the city as an extra word in the review, and thus it will automatically be included in the model! Check to see whether it increased or decreased the AUC of your **best model**.

In [21]:
mcd_na['review_city'] = mcd_na.review.str.cat(mcd_na.city, na_rep= 'na', sep= " ")

In [22]:
X = mcd_na.review_city
y = mcd_na.rude

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
print(len(X_train))

1103


In [24]:
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

In [25]:
nb.fit(X_train_dtm,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [26]:
nb_pred_class = nb.predict(X_test_dtm)

In [27]:
vect = CountVectorizer(max_df= .2)
tokenize_test(vect)

Features:  7370
Accuracy:  0.771739130435


## Task 8 (Challenge)

The **policies_violated:confidence** column may be useful, since it essentially represents a measurement of the training data quality. Let's see whether we can improve the AUC by only training the model using higher-quality rows!

To accomplish this, your first sub-task is to **calculate the mean confidence score for each row**, and then store those mean scores in a new column. For example, the confidence scores for the first row are `1.0\r\n0.6667\r\n0.6667`, so you should calculate a mean of `0.7778`. Here are the suggested steps:

1. Using the [`Series.str.split()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.split.html) method, convert the **policies_violated:confidence** column into lists of one or more "confidence scores". Save the results as a new DataFrame column called **confidence_list**.
2. Define a function that calculates the mean of a list of numbers, and pass that function to the [`Series.apply()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.apply.html) method of the **confidence_list** column. That will calculate the mean confidence score for each row. Save those scores in a new DataFrame column called **confidence_mean**.
    - **Hint:** [How do I apply a function to a pandas Series or DataFrame?](https://www.youtube.com/watch?v=P_q0tkYqvSk&list=PL5-da3qGB5ICCsgW1MxlZ0Hq8LL5U3u9y&index=30) explains how to use the `Series.apply()` method.

In [28]:
# Convert to float and calculate the mean of a list 
def mn_cfd(lst):
    
    lst = [float(i) for i in lst]
      
    return np.mean(lst)
    

In [29]:
mcd_na['confidence_list'] = mcd_na['policies_violated:confidence'].str.split()
mcd_na['confidence_mean'] = mcd_na.confidence_list.apply(mn_cfd)
mcd_na.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review,Unnamed: 10,rude,review_city,confidence_list,confidence_mean
0,679455653,False,finalized,3,2/21/15 0:36,RudeService\r\nOrderProblem\r\nFilthy,1.0\r\n0.6667\r\n0.6667,Atlanta,NaN,"I'm not a huge mcds lover, but I've been to be...",NaN,True,"I'm not a huge mcds lover, but I've been to be...","[1.0, 0.6667, 0.6667]",0.7778
1,679455654,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,Terrible customer service. ŒæI came in at 9:30...,NaN,True,Terrible customer service. ŒæI came in at 9:30...,[1],1.0000
2,679455655,False,finalized,3,2/21/15 0:26,SlowService\r\nOrderProblem,1.0\r\n1.0,Atlanta,NaN,"First they ""lost"" my order, actually they gave...",NaN,False,"First they ""lost"" my order, actually they gave...","[1.0, 1.0]",1.0000
3,679455656,False,finalized,3,2/21/15 0:27,na,0.6667,Atlanta,NaN,I see I'm not the only one giving 1 star. Only...,NaN,False,I see I'm not the only one giving 1 star. Only...,[0.6667],0.6667
4,679455657,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,"Well, it's McDonald's, so you know what the fo...",NaN,True,"Well, it's McDonald's, so you know what the fo...",[1],1.0000


Your second sub-task is to **remove lower-quality rows from the training set**, and then repeat the model building and evaluation process. Here are the suggested steps:

1. Remove all rows from X_train and y_train that have a **confidence_mean lower than 0.75**. Check their shapes before and after to confirm that you removed about 300 rows.
2. Use the **`tokenize_test()`** function to check whether filtering the training data increased or decreased the AUC of your **best model**.
    - **Hint:** Even though X_train and y_train are separate from the mcd DataFrame, they can still be filtered using a boolean Series generated from mcd because all three objects share the same index.
    - **Note:** It's important that we don't remove any rows from the testing set (X_test and y_test), because the testing set should be representative of the real-world data we will encounter in the future (which will contain both high-quality and low-quality rows).

In [30]:
X_train = X_train[mcd_na.confidence_mean >= 0.75]
y_train = y_train[mcd_na.confidence_mean >= 0.75]

In [31]:
vect = CountVectorizer()
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

In [32]:
nb.fit(X_train_dtm,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [33]:
vect = CountVectorizer(min_df= 3)
tokenize_test(vect)

Features:  2007
Accuracy:  0.725543478261


## Task 9 (Challenge)

New comments have been submitted to the McDonald's website, and you need to **score them with the likelihood** that they are referring to rude service.

1. Before making predictions on out-of-sample data, it is important to re-train your model on all relevant data using the tuning parameters and preprocessing steps that produced the best AUC above.
    - In other words, X should be defined using either **all rows** or **only those rows with a confidence_mean of at least 0.75**, whichever produced a better AUC above.
    - X should refer to either the **review column** or the **review_city column**, whichever produced a better AUC above.
    - CountVectorizer should be instantiated with the **tuning parameters** that produced the best AUC above.
    - **`train_test_split()`** should not be used during this process.
2. Build a document-term matrix (from X) called **X_dtm**, and examine its shape.
3. Read the new comments stored in **`mcdonalds_new.csv`** into a DataFrame called **new_comments**, and examine it.
4. If your model uses a **review_city** column, create that column in the new_comments DataFrame. (Otherwise, skip this step.)
5. Build a document_term matrix (from the **new_comments** DataFrame) called **new_dtm**, and examine its shape.
6. Train your best model (Naive Bayes or logistic regression) using **X_dtm** and **y**.
7. Predict the "rude probability" for each comment in **new_dtm**, and store the probabilities in an object called **new_pred_prob**.
8. Print the **full text** for each new comment alongside its **"rude probability"**. (You may need to [increase the max_colwidth](https://www.youtube.com/watch?v=yiO43TQ4xvc&list=PL5-da3qGB5ICCsgW1MxlZ0Hq8LL5U3u9y&index=28) to see the full text.) Examine the results, and comment on how well you think the model performed!